<a href="https://colab.research.google.com/github/jameschapman19/cca_zoo/blob/master/cca_zoo_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install cca-zoo --upgrade
!pip install scipy --upgrade

Requirement already up-to-date: cca-zoo in /usr/local/lib/python3.7/dist-packages (1.2.0)
Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (1.6.2)


In [7]:
# Imports
import numpy as np
from cca_zoo import wrappers
from cca_zoo import data
import itertools
import os
import matplotlib.pyplot as plt
from torch.utils.data import Subset
from torch import optim

# Load MNIST Data
os.chdir('..')
N = 500
dataset = data.Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=True)
ids = np.arange(min(2 * N, len(dataset)))
np.random.shuffle(ids)
train_ids, val_ids = np.array_split(ids, 2)
val_dataset = Subset(dataset, val_ids)
train_dataset = Subset(dataset, train_ids)
test_dataset = data.Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=False)
test_ids = np.arange(min(N, len(test_dataset)))
np.random.shuffle(test_ids)
test_dataset = Subset(test_dataset, test_ids)
train_view_1, train_view_2, train_rotations, train_OH_labels, train_labels = train_dataset.dataset.to_numpy(
    train_dataset.indices)
val_view_1, val_view_2, val_rotations, val_OH_labels, val_labels = val_dataset.dataset.to_numpy(val_dataset.indices)
test_view_1, test_view_2, test_rotations, test_OH_labels, test_labels = test_dataset.dataset.to_numpy(
    test_dataset.indices)

# Settings

# The number of latent dimensions across models
latent_dims = 2
# The number of folds used for cross-validation/hyperparameter tuning
cv_folds = 5
# For running hyperparameter tuning in parallel (0 if not)
jobs = 2
# Number of iterations for iterative algorithms
max_iter = 2
# number of epochs for deep models
epochs = 50

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:936: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


# Canonical Correlation Analysis

In [8]:
"""
### Linear CCA by eigendecomposition
"""
linear_cca = wrappers.CCA(latent_dims=latent_dims)

linear_cca.fit(train_view_1, train_view_2)

linear_cca_results = np.stack(
    (linear_cca.train_correlations[0, 1], linear_cca.predict_corr(test_view_1, test_view_2)[0, 1]))

"""
### Linear CCA by alternating least squares (can pass more than 2 views)
"""

linear_cca_als = wrappers.CCA_ALS(latent_dims=latent_dims)

linear_cca_als.fit(train_view_1, train_view_2)

linear_cca_als_results = np.stack(
    (linear_cca_als.train_correlations[0, 1], linear_cca_als.predict_corr(test_view_1, test_view_2)[0, 1]))

# Partial Least Squares


In [9]:
"""
### PLS with scikit-learn (only permits 2 views)
"""
pls = wrappers.PLS(latent_dims=latent_dims)

pls.fit(train_view_1, train_view_2)

pls_results = np.stack(
    (pls.train_correlations[0, 1], pls.predict_corr(test_view_1, test_view_2)[0, 1]))

# Extension to multiple views



In [10]:
"""
### (Regularized) Generalized CCA(can pass more than 2 views)
"""
# small ammount of regularisation added since data is not full rank
c=[0.5,0.5,0.5]

gcca = wrappers.GCCA(latent_dims=latent_dims,c=c)

gcca.fit(train_view_1, train_view_2,train_view_1)

gcca_results = np.stack((gcca.train_correlations[0, 1], gcca.predict_corr(test_view_1, test_view_2)[0, 1]))

"""
### (Regularized) Multiset CCA(can pass more than 2 views)
"""

mcca = wrappers.MCCA(latent_dims=latent_dims, c=c)

mcca.fit(train_view_1, train_view_2,train_view_1)

mcca_results = np.stack((mcca.train_correlations[0, 1], mcca.predict_corr(test_view_1, test_view_2)[0, 1]))

"""
### Multiset CCA by alternating least squares
"""
mcca_als = wrappers.CCA_ALS(latent_dims=latent_dims, max_iter=max_iter)

mcca_als.fit(train_view_1, train_view_2,train_view_1)

mcca_als_results = np.stack(
    (mcca_als.train_correlations[0, 1], mcca_als.predict_corr(test_view_1, test_view_2)[0, 1]))

"""
### Multiset PLS by alternating least squares
"""
mcca_pls = wrappers.PLS(latent_dims=latent_dims, max_iter=max_iter)

mcca_pls.fit(train_view_1, train_view_2,train_view_1)

mcca_pls_results = np.stack(
    (mcca_als.train_correlations[0, 1], mcca_pls.predict_corr(test_view_1, test_view_2)[0, 1]))

# Weighted GCCA/Missing Observation GCCA

In [12]:
#observation_matrix
K = np.ones((3, N))
K[0, 200:] = 0
K[1, :100] = 0

#view weights
view_weights=[1,2,1.2]

c=[0.5,0.5,0.5]

gcca = wrappers.GCCA(latent_dims=latent_dims,c=c,view_weights=view_weights)

gcca.fit(train_view_1, train_view_2,train_view_1,K=K)

gcca_results = np.stack((gcca.train_correlations[0, 1], gcca.predict_corr(test_view_1, test_view_2)[0, 1]))

# Rgularised CCA solutions based on alternating minimisation/alternating least squares

We implement Witten's penalized matrix decomposition form of sparse CCA using 'pmd'

We implement Waaijenborg's penalized CCA using elastic net using 'elastic'

We implement Mai's sparse CCA using 'scca'

Furthermore, any of these methods can be extended to multiple views. Witten describes this method explicitly.

In [13]:
"""
### Ridge CCA (can pass more than 2 views)
"""
c1 = [0.1, 0.3, 0.7, 0.9]
c2 = [0.1, 0.3, 0.7, 0.9]
param_candidates = {'c': list(itertools.product(c1, c2))}

ridge = wrappers.rCCA(latent_dims=latent_dims).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True, jobs=jobs,
    plot=True)

ridge_results = np.stack((ridge.train_correlations[0, 1, :], ridge.predict_corr(test_view_1, test_view_2)[0, 1, :]))

"""
### Sparse CCA (Penalized Matrix Decomposition) (can pass more than 2 views)
"""

# PMD
c1 = [1, 3, 7, 9]
c2 = [1, 3, 7, 9]
param_candidates = {'c': list(itertools.product(c1, c2))}

pmd = wrappers.PMD(latent_dims=latent_dims, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True, jobs=jobs,
    plot=True)

pmd_results = np.stack((pmd.train_correlations[0, 1, :], pmd.predict_corr(test_view_1, test_view_2)[0, 1, :]))

"""
### Sparse CCA (can pass more than 2 views)
"""

# Sparse CCA
c1 = [0.00001, 0.0001]
c2 = [0.00001, 0.0001]
param_candidates = {'c': list(itertools.product(c1, c2))}

scca = wrappers.SCCA(latent_dims=latent_dims, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True,
    jobs=jobs, plot=True)

scca_results = np.stack(
    (scca.train_correlations[0, 1, :], scca.predict_corr(test_view_1, test_view_2)[0, 1, :]))


"""
### Elastic CCA (can pass more than 2 views)
"""

# Elastic CCA
c1 = [0.001, 0.0001]
c2 = [0.001, 0.0001]
l1_1 = [0.01, 0.1]
l1_2 = [0.01, 0.1]
param_candidates = {'c': list(itertools.product(c1, c2)), 'l1_ratio': list(itertools.product(l1_1, l1_2))}

elastic = wrappers.ElasticCCA(latent_dims=latent_dims,
                              max_iter=max_iter).gridsearch_fit(train_view_1,
                                                                train_view_2,
                                                                param_candidates=param_candidates,
                                                                folds=cv_folds,
                                                                verbose=True,
                                                                jobs=jobs,
                                                                plot=True)

elastic_results = np.stack(
    (elastic.train_correlations[0, 1, :], elastic.predict_corr(test_view_1, test_view_2)[0, 1, :]))

cross validation
number of folds:  5
Best score :  1.1580229366975092
Standard deviation :  0.0744296519833356
{'c': (0.9, 0.9)}
cross validation
number of folds:  5
Best score :  1.2096107222263128
Standard deviation :  0.07042732486816454
{'c': (9, 9)}
cross validation
number of folds:  5
Best score :  0.7671002196854806
Standard deviation :  0.11205946080012376
{'c': (0.0001, 1e-05)}
cross validation
number of folds:  5
Best score :  0.8174683020077215
Standard deviation :  0.027509691263146254
{'c': (0.001, 0.001), 'l1_ratio': (0.1, 0.01)}


# Kernel CCA

In [14]:
"""
### Kernel CCA

Similarly, we can use kernel CCA methods with [method='kernel']

We can use different kernels and their associated parameters in a similar manner to before
- regularized linear kernel CCA: parameters :  'kernel'='linear', 0<'c'<1
- polynomial kernel CCA: parameters : 'kernel'='poly', 'degree', 0<'c'<1
- gaussian rbf kernel CCA: parameters : 'kernel'='gaussian', 'sigma', 0<'c'<1
"""
# %%
# r-kernel cca
c1 = [0.9, 0.99]
c2 = [0.9, 0.99]

param_candidates = {'kernel': ['linear'], 'c': list(itertools.product(c1, c2))}

kernel_reg = wrappers.KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                                   folds=cv_folds,
                                                                   param_candidates=param_candidates,
                                                                   verbose=True, jobs=jobs,
                                                                   plot=True)
kernel_reg_results = np.stack((
    kernel_reg.train_correlations[0, 1, :],
    kernel_reg.predict_corr(test_view_1, test_view_2)[0, 1, :]))

# kernel cca (poly)
param_candidates = {'kernel': ['poly'], 'degree': [2, 3], 'c': list(itertools.product(c1, c2))}

kernel_poly = wrappers.KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                                    folds=cv_folds,
                                                                    param_candidates=param_candidates,
                                                                    verbose=True, jobs=jobs,
                                                                    plot=True)

kernel_poly_results = np.stack((
    kernel_poly.train_correlations[0, 1, :],
    kernel_poly.predict_corr(test_view_1, test_view_2)[0, 1, :]))

# kernel cca (gaussian)
param_candidates = {'kernel': ['rbf'], 'sigma': [1e+1, 1e+2, 1e+3], 'c': list(itertools.product(c1, c2))}

kernel_gaussian = wrappers.KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                                        folds=cv_folds,
                                                                        param_candidates=param_candidates,
                                                                        verbose=True, jobs=jobs,
                                                                        plot=True)

kernel_gaussian_results = np.stack((
    kernel_gaussian.train_correlations[0, 1, :],
    kernel_gaussian.predict_corr(test_view_1, test_view_2)[0, 1, :]))

cross validation
number of folds:  5
Best score :  1.4173160345413034
Standard deviation :  0.07843160389828484
{'kernel': 'linear', 'c': (0.99, 0.99)}
cross validation
number of folds:  5
Best score :  1.0184435578673425
Standard deviation :  0.1203279634354833
{'kernel': 'poly', 'degree': 3, 'c': (0.9, 0.9)}
cross validation
number of folds:  5
Best score :  1.157831396516231
Standard deviation :  0.11527570837893727
{'kernel': 'rbf', 'sigma': 100.0, 'c': (0.99, 0.9)}


# Deep CCA

DCCA can be optimized using Andrew's original tracenorm objective or Wang's DCCA by nonlinear orthogonal iterations using the argument als=True.

In [15]:
"""
### Deep Learning

We also have deep CCA methods (and autoencoder variants)
- Deep CCA (DCCA)
- Deep Canonically Correlated Autoencoders (DCCAE)

We introduce a Config class from configuration.py. This contains a number of default settings for running DCCA.

"""
from cca_zoo import deepwrapper, objectives, dcca, deep_models

# %%
# DCCA
print('DCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_model = deepwrapper.DeepWrapper(dcca_model)

dcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dcca_results = np.stack((dcca_model.train_correlations[0, 1], dcca_model.predict_corr(test_dataset)[0, 1]))


# DCCA_NOI
# Note that als=True
print('DCCA by non-linear orthogonal iterations')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_noi_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], als=True)

dcca_noi_model = deepwrapper.DeepWrapper(dcca_noi_model)

dcca_noi_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dcca_noi_results = np.stack(
    (dcca_noi_model.train_correlations[0, 1], dcca_noi_model.predict_corr(test_dataset)[0, 1]))

DCCA
total parameters:  201476


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:936: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


====> Epoch: 1 Average train loss: -0.2773
====> Epoch: 1 Average val loss: -0.8487
Min loss -0.85
====> Epoch: 2 Average train loss: -0.6858
====> Epoch: 2 Average val loss: -1.1395
Min loss -1.14
====> Epoch: 3 Average train loss: -1.0905
====> Epoch: 3 Average val loss: -1.2920
Min loss -1.29
====> Epoch: 4 Average train loss: -1.1826
====> Epoch: 4 Average val loss: -1.2559
====> Epoch: 5 Average train loss: -1.3236
====> Epoch: 5 Average val loss: -1.3220
Min loss -1.32
====> Epoch: 6 Average train loss: -1.2880
====> Epoch: 6 Average val loss: -1.3762
Min loss -1.38
====> Epoch: 7 Average train loss: -1.2604
====> Epoch: 7 Average val loss: -1.3424
====> Epoch: 8 Average train loss: -1.3732
====> Epoch: 8 Average val loss: -1.3888
Min loss -1.39
====> Epoch: 9 Average train loss: -1.3910
====> Epoch: 9 Average val loss: -1.4136
Min loss -1.41
====> Epoch: 10 Average train loss: -1.3582
====> Epoch: 10 Average val loss: -1.4313
Min loss -1.43
====> Epoch: 11 Average train loss: -1

# DCCA with custom optimizers and schedulers

In [16]:
# DCCA
optimizers = [optim.Adam(encoder_1.parameters(), lr=1e-4), optim.Adam(encoder_2.parameters(), lr=1e-4)]
schedulers = [optim.lr_scheduler.CosineAnnealingLR(optimizers[0], 1),
              optim.lr_scheduler.ReduceLROnPlateau(optimizers[1])]
dcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2],
                                objective=objectives.CCA, optimizers=optimizers, schedulers=schedulers)
# hidden_layer_sizes are shown explicitly but these are also the defaults
dcca_model = deepwrapper.DeepWrapper(dcca_model)
dcca_model.fit(train_dataset, val_dataset=val_dataset,epochs=20)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:936: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


total parameters:  201476
====> Epoch: 1 Average train loss: -0.0976
====> Epoch: 1 Average val loss: -0.1331
Min loss -0.13
====> Epoch: 2 Average train loss: -0.1345
====> Epoch: 2 Average val loss: -0.1524
Min loss -0.15
====> Epoch: 3 Average train loss: -0.1413
====> Epoch: 3 Average val loss: -0.1598
Min loss -0.16
====> Epoch: 4 Average train loss: -0.1507
====> Epoch: 4 Average val loss: -0.1572
====> Epoch: 5 Average train loss: -0.1610
====> Epoch: 5 Average val loss: -0.2026
Min loss -0.20
====> Epoch: 6 Average train loss: -0.1699
====> Epoch: 6 Average val loss: -0.2012
====> Epoch: 7 Average train loss: -0.2136
====> Epoch: 7 Average val loss: -0.2427
Min loss -0.24
====> Epoch: 8 Average train loss: -0.2145
====> Epoch: 8 Average val loss: -0.2604
Min loss -0.26
====> Epoch: 9 Average train loss: -0.2642
====> Epoch: 9 Average val loss: -0.2720
Min loss -0.27
====> Epoch: 10 Average train loss: -0.2478
====> Epoch: 10 Average val loss: -0.2952
Min loss -0.30
====> Epoch:

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


DeepWrapper(device='cpu',
            model=DCCA(
  (encoders): ModuleList(
    (0): Encoder(
      (layers): Sequential(
        (0): Sequential(
          (0): Linear(in_features=784, out_features=128, bias=True)
          (1): ReLU()
        )
      )
      (fc): Linear(in_features=128, out_features=2, bias=True)
    )
    (1): Encoder(
      (layers): Sequential(
        (0): Sequential(
          (0): Linear(in_features=784, out_features=128, bias=True)
          (1): ReLU()
        )
      )
      (fc): Linear(in_features=128, out_features=2, bias=True)
    )
  )
),
            tensorboard=False, tensorboard_tag=None)

# DGCCA and DMCCA for more than 2 views

The only change we need to make is to the objective argument to perform DGCCA and DMCCA.

In [17]:
# DGCCA
print('DGCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dgcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.GCCA)

dgcca_model = deepwrapper.DeepWrapper(dgcca_model)

dgcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dgcca_results = np.stack(
    (dgcca_model.train_correlations[0, 1], dgcca_model.predict_corr(test_dataset)[0, 1]))

# DMCCA
print('DMCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dmcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.MCCA)

dmcca_model = deepwrapper.DeepWrapper(dmcca_model)

dmcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dmcca_results = np.stack(
    (dmcca_model.train_correlations[0, 1], dmcca_model.predict_corr(test_dataset)[0, 1]))

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:936: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


DGCCA
total parameters:  201476
====> Epoch: 1 Average train loss: -0.2276
====> Epoch: 1 Average val loss: -1.0637
Min loss -1.06
====> Epoch: 2 Average train loss: -0.9894
====> Epoch: 2 Average val loss: -1.2232
Min loss -1.22
====> Epoch: 3 Average train loss: -1.2018
====> Epoch: 3 Average val loss: -1.3136
Min loss -1.31
====> Epoch: 4 Average train loss: -1.2822
====> Epoch: 4 Average val loss: -1.4474
Min loss -1.45
====> Epoch: 5 Average train loss: -1.3327
====> Epoch: 5 Average val loss: -1.4549
Min loss -1.45
====> Epoch: 6 Average train loss: -1.4326
====> Epoch: 6 Average val loss: -1.4645
Min loss -1.46
====> Epoch: 7 Average train loss: -1.4150
====> Epoch: 7 Average val loss: -1.4986
Min loss -1.50
====> Epoch: 8 Average train loss: -1.4307
====> Epoch: 8 Average val loss: -1.5470
Min loss -1.55
====> Epoch: 9 Average train loss: -1.4586
====> Epoch: 9 Average val loss: -1.5280
====> Epoch: 10 Average train loss: -1.5138
====> Epoch: 10 Average val loss: -1.5065
====> 

# Deep Canonically Correlated Autoencoders
We need to add decoders in order to model deep canonically correlated autoencoders and we also use the DCCAE class which inherits from DCCA

In [19]:
from cca_zoo import dccae

# DCCAE
print('DCCAE')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
decoder_1 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784)
decoder_2 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784)
dccae_model = dccae.DCCAE(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2])

dccae_model = deepwrapper.DeepWrapper(dccae_model)

#can also pass a tuple of numpy arrays
dccae_model.fit((train_view_1, train_view_2), epochs=epochs)

dccae_results = np.stack(
    (dccae_model.train_correlations[0, 1], dccae_model.predict_corr(test_dataset)[0, 1]))

DCCAE
total parameters:  404516
====> Epoch: 1 Average train loss: 55446.6602
====> Epoch: 1 Average val loss: 11446.3047
Min loss 11446.30
====> Epoch: 2 Average train loss: 46537.3086
====> Epoch: 2 Average val loss: 9816.6934
Min loss 9816.69
====> Epoch: 3 Average train loss: 39998.5781
====> Epoch: 3 Average val loss: 8677.6221
Min loss 8677.62
====> Epoch: 4 Average train loss: 35431.3008
====> Epoch: 4 Average val loss: 7906.2661
Min loss 7906.27
====> Epoch: 5 Average train loss: 32344.1191
====> Epoch: 5 Average val loss: 7400.0356
Min loss 7400.04
====> Epoch: 6 Average train loss: 30322.6309
====> Epoch: 6 Average val loss: 7086.2070
Min loss 7086.21
====> Epoch: 7 Average train loss: 29072.0000
====> Epoch: 7 Average val loss: 6908.5840
Min loss 6908.58
====> Epoch: 8 Average train loss: 28364.0977
====> Epoch: 8 Average val loss: 6820.3354
Min loss 6820.34
====> Epoch: 9 Average train loss: 28009.3301
====> Epoch: 9 Average val loss: 6783.6299
Min loss 6783.63
====> Epoch:

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:936: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


# Deep Variational CCA

In [20]:
"""
### Deep Variational Learning
Finally we have Deep Variational CCA methods.
- Deep Variational CCA (DVCCA)
- Deep Variational CCA - private (DVVCA_p)

These are both implemented by the DVCCA class with private=True/False and both_encoders=True/False. If both_encoders,
the encoder to the shared information Q(z_shared|x) is modelled for both x_1 and x_2 whereas if both_encoders is false
it is modelled for x_1 as in the paper
"""
from cca_zoo import dvcca

# %%
# DVCCA (technically bi-DVCCA)
print('DVCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
decoder_2 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
dvcca_model = dvcca.DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2],
                          private=False)

dvcca_model = deepwrapper.DeepWrapper(dvcca_model)

dvcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dvcca_model_results = np.stack(
    (dvcca_model.train_correlations[0, 1], dvcca_model.predict_corr(test_dataset)[0, 1]))

# DVCCA_private (technically bi-DVCCA_private)
print('DVCCA_private')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = deep_models.Decoder(latent_dims=latent_dims * 2, feature_size=784, norm_output=True)
decoder_2 = deep_models.Decoder(latent_dims=latent_dims * 2, feature_size=784, norm_output=True)
dvccap_model = dvcca.DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2],
                           private_encoders=[private_encoder_1, private_encoder_2], private=True)

dvccap_model = deepwrapper.DeepWrapper(dvccap_model)

dvccap_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dvccap_model_results = np.stack(
    (dvccap_model.train_correlations[0, 1], dvccap_model.predict_corr(test_dataset)[0, 1]))

DVCCA
total parameters:  405032


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:936: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


====> Epoch: 1 Average train loss: 1102.7065
====> Epoch: 1 Average val loss: 1080.3849
Min loss 1080.38
====> Epoch: 2 Average train loss: 1081.1470
====> Epoch: 2 Average val loss: 1060.3260
Min loss 1060.33
====> Epoch: 3 Average train loss: 1060.9924
====> Epoch: 3 Average val loss: 1041.2097
Min loss 1041.21
====> Epoch: 4 Average train loss: 1042.2260
====> Epoch: 4 Average val loss: 1024.2603
Min loss 1024.26
====> Epoch: 5 Average train loss: 1026.2439
====> Epoch: 5 Average val loss: 1009.3326
Min loss 1009.33
====> Epoch: 6 Average train loss: 1009.9720
====> Epoch: 6 Average val loss: 994.4794
Min loss 994.48
====> Epoch: 7 Average train loss: 994.9193
====> Epoch: 7 Average val loss: 981.2676
Min loss 981.27
====> Epoch: 8 Average train loss: 982.4504
====> Epoch: 8 Average val loss: 969.2190
Min loss 969.22
====> Epoch: 9 Average train loss: 972.2469
====> Epoch: 9 Average val loss: 956.1982
Min loss 956.20
====> Epoch: 10 Average train loss: 958.7764
====> Epoch: 10 Avera

# Convolutional Deep CCA (and using other architectures)
We provide a standard CNN encoder and decoder but users can build their own encoders and decoders by inheriting BaseEncoder and BaseDecoder for seamless integration with the pipeline

In [25]:
print('Convolutional DCCA')
encoder_1 = deep_models.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
encoder_2 = deep_models.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
dcca_conv_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_conv_model = deepwrapper.DeepWrapper(dcca_conv_model)

# to change the models used change the cfg.encoder_models. We implement a CNN_Encoder and CNN_decoder as well
# as some based on brainnet architecture in cca_zoo.deep_models. Equally you could pass your own encoder/decoder models

dcca_conv_model.fit((train_view_1.reshape((-1, 1, 28, 28)), train_view_2.reshape((-1, 1, 28, 28))), epochs=epochs)

dcca_conv_results = np.stack(
    (dcca_conv_model.train_correlations[0, 1], dcca_conv_model.predict_corr((test_view_1.reshape((-1, 1, 28, 28)),
                                                                            test_view_2.reshape(
                                                                                (-1, 1, 28, 28))))[0, 1]))

Convolutional DCCA
total parameters:  9568
====> Epoch: 1 Average train loss: -0.0993
====> Epoch: 1 Average val loss: -0.2911
Min loss -0.29
====> Epoch: 2 Average train loss: -0.5038
====> Epoch: 2 Average val loss: -0.4841
Min loss -0.48
====> Epoch: 3 Average train loss: -0.8328
====> Epoch: 3 Average val loss: -0.6386
Min loss -0.64
====> Epoch: 4 Average train loss: -1.0509
====> Epoch: 4 Average val loss: -0.7580
Min loss -0.76
====> Epoch: 5 Average train loss: -1.1914
====> Epoch: 5 Average val loss: -0.8454
Min loss -0.85
====> Epoch: 6 Average train loss: -1.2920
====> Epoch: 6 Average val loss: -0.9087
Min loss -0.91
====> Epoch: 7 Average train loss: -1.3716
====> Epoch: 7 Average val loss: -0.9543
Min loss -0.95
====> Epoch: 8 Average train loss: -1.4388
====> Epoch: 8 Average val loss: -0.9866
Min loss -0.99
====> Epoch: 9 Average train loss: -1.4972
====> Epoch: 9 Average val loss: -1.0081
Min loss -1.01
====> Epoch: 10 Average train loss: -1.5481
====> Epoch: 10 Averag

# Generate Some Plots

In [26]:
"""
### Make results plot to compare methods
"""
# %%

all_results = np.stack(
    [linear_cca_results, gcca_results, mcca_results, pls_results, pmd_results, elastic_results,
     scca_results, kernel_reg_results, kernel_poly_results,
     kernel_gaussian_results, dcca_results, dgcca_results, dmcca_results, dvcca_model_results,
     dcca_conv_results],
    axis=0)
all_labels = ['linear', 'gcca', 'mcca', 'pls', 'pmd', 'elastic', 'scca', 'linear kernel', 'polynomial kernel',
              'gaussian kernel', 'deep CCA', 'deep generalized CCA', 'deep multiset CCA', 'deep VCCA',
              'deep convolutional cca']

from cca_zoo import plot_utils

plot_utils.plot_results(all_results, all_labels)
plt.show()